In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re
import seaborn as sns
import QuantLib as ql

from bgs.bgs_utils import clean_date, clean_percentage
from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices
from bgs.gilt_analytics import gilt_yield, yield_series
from bgs.linker_analytics import linker_real_yield

### British Gilt Survey Amounts Table

In [ ]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')
conv.columns = [np.int64(x.strip()) for x in conv.columns]

old = tables['Calculated indexed nominal Old-style']
old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')
old.columns = [np.int64(x.strip()) for x in old.columns]

new = tables['Calculated indexed nominal New-style']
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)
new.index = pd.Index([clean_date(x) for x in list(new.index)]).to_period('M').to_timestamp('M')
new.columns = [np.int64(x.strip()) for x in new.columns]

new_no_idx = tables['Index-linked New-style']
new_no_idx.replace("Redeemed", "", inplace=True)
new_no_idx = new_no_idx.apply(pd.to_numeric, errors='coerce')
new_no_idx.fillna(0, inplace=True)
new_no_idx.index = pd.Index([clean_date(x) for x in list(new_no_idx.index)]).to_period('M').to_timestamp('M')
new_no_idx.columns = [np.int64(x.strip()) for x in new_no_idx.columns]

### British Gilts Survey Month End Prices

In [ ]:
price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)
price_df.columns = [np.int64(x) for x in price_df.columns]

### British Gilts Survey Details Table (Bond Static Data, Coupon Maturity etc.) 

In [ ]:
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)
conv_details['Sequence'] = conv_details['Sequence'].apply(lambda x: np.int64(x))

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)
new_details['Sequence'] = new_details['Sequence'].apply(lambda x: np.int64(x))

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)
old_details['Sequence'] = old_details['Sequence'].apply(lambda x: np.int64(x))

strip_details = details['Strips']
strip_details['%'] = 0.0
strip_details['Sequence'] = strip_details['Sequence'].apply(lambda x: np.int64(x))

date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
for date_var in date_variables:
    conv_details[date_var] = conv_details[date_var].apply(clean_date)
    old_details[date_var] = old_details[date_var].apply(clean_date)
    new_details[date_var] = new_details[date_var].apply(clean_date)
strip_details['Latest redemption date'] = strip_details['Latest redemption date'].apply(clean_date)